In [1]:
# Okay, let's preprocess all data

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

2025-07-28 09:11:57.053459: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 09:11:58.668720: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-28 09:11:58.668783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-28 09:11:58.810916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 09:11:59.070096: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# What all do we have? We have the geenrated features, and the densities defined for all the test blocks.

In [3]:
# Do the data prep for one block, and define a function for the rest

In [4]:
# densities are here
block_0103_density_file = np.load(os.path.join("data/densities_non_overlapping/", "all_densities_block_0103.npy"))

In [5]:
block_0103_density_file.shape

(910, 20, 1)

In [6]:
# Where are the feature files?

In [10]:
# for test time points
block_0103_feature_file_test_points = np.load(os.path.join("data/predicted_sequences_from_stage_1_non_overlapping/model_1/", "pred_values_blk_0103.npy"))

In [11]:
block_0103_feature_file_test_points.shape

(910, 7, 32)

In [12]:
# for train time points
block_0103_feature_file_train_points = np.load(os.path.join("data/train_features_non_overlapping/", "train_features_block_0103.npy"))

In [13]:
block_0103_feature_file_train_points.shape

(910, 13, 32)

In [14]:
# Stack the featrue files together?